In [1]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import requests

I am importing everything we used in the lab.

In [2]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
source = requests.get(url).text
soup= BeautifulSoup(source, "lxml")
table = soup.find('table')

In [5]:
df=pd.read_html(str(table))[0]

Drop all rows that have Not assigned for both the Borough and the Neighbourhood columns.

In [6]:
rowdrop = df[(df['Borough']=='Not assigned')&(df['Neighbourhood']=='Not assigned')].index
df.drop(rowdrop, inplace=True)

Change all not assigned values to the name of the Borough

In [7]:
df.loc[df['Neighbourhood']==('Not assigned'),'Neighbourhood']=df['Borough']

Merge all postal codes that have two neighbourhoods

In [8]:
df=df.groupby(['Postcode','Borough'],sort=False).agg(lambda x:','.join(x))

In [9]:
df.reset_index(inplace=True)

In [10]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [11]:
df.shape

(103, 3)




This is the beginning of part two of the assignment. I am going to find the longitude and latitude of all the postol codes





In [11]:
#!conda install -c conda-forge geocoder -y  I am not running this everytime I restart so I committed it out.
#import geocoder # import geocoder

Solving environment: done

# All requested packages already installed.



In [ ]:
#postal_code=df['Postcode']

#lat_lng_coords = None

#while(lat_lng_coords is None):
#  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#  lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

I left the above cell commited out because it would only run for an hour and not give results

n=211

postal_code = df['Postcode']
for i in range(n):

  print("row:",i)

  lat_lng_coords = None

  while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code[i]))
      lat_lng_coords = g.latlng
      latitude.append(lat_lng_coords[0])
      longitude.append(lat_lng_coords[1])

The above was a suggustion from the dissucsion. It also ran for an hour before I stopped it. The only result I am getting is 
row 0
I will use the provided CSV to finish the assignment.

In [12]:
latlon_df=pd.read_csv('http://cocl.us/Geospatial_data')

In [13]:
latlon_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
latlon_df=latlon_df.rename(columns=({'Postal Code':'Postcode'}))

In [15]:
df_loc=pd.merge(df,latlon_df,on='Postcode')

In [16]:
df_loc.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [17]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(df_loc['Borough'].unique()),df_loc.shape[0]))

The dataframe has 11 boroughs and 103 neighborhoods.


This is the beginning of part 3 of the assignment

I will follow the week 3 lab.

First I will get the location of Toronto then I will create a map with all the Boroughs.

In [18]:
address='Toronto, Ontario'
geolocator= Nominatim(user_agent="to_explorer")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('The geograpical coordinate of Toronoto are {},{}.'.format(latitude,longitude))

The geograpical coordinate of Toronoto are 43.653963,-79.387207.


In [19]:
map_toronto=folium.Map(location=[latitude, longitude], zoom_start=10)

In [20]:
#add markers for all the boroughs
for lat, lng, postcode, neighborhood, borough in zip(df_loc['Latitude'],df_loc['Longitude'],df_loc['Postcode'],df_loc['Neighbourhood'],df_loc['Borough']):
    label='{},{},{}'.format(postcode,neighborhood,borough)
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)


I choose to have all the neighborhoods listed before the borough so order in each circle is post code then neighborhood then borough.

In [21]:
map_toronto

In [22]:
df_loc.Borough.unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

I will only use the Boroughs that have Toronto in the name. I am creating a new data frame with just the Boroughs Downtown Toronto, East Toronto, West Toronto, and Central Toronto. 

In [23]:
toronto_df=df_loc[df_loc['Borough'].isin(['Central Toronto','East Toronto','West Toronto','Downtown Toronto'])]

In [24]:
toronto_df.reset_index(drop=True,inplace=True)
toronto_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


I will create a new map with just these Boroughs. Since I already have the location of Toronoto and a map set up I will just switch the dataframe from the previous code.

In [25]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(toronto_df['Borough'].unique()),toronto_df.shape[0]))

The dataframe has 4 boroughs and 38 neighborhoods.


In [26]:
map_toronto2=folium.Map(location=[latitude, longitude], zoom_start=11)
#add markers for all the boroughs
for lat, lng, postcode, neighborhood, borough in zip(toronto_df['Latitude'],toronto_df['Longitude'],toronto_df['Postcode'],toronto_df['Neighbourhood'],toronto_df['Borough']):
    label='{},{},{}'.format(postcode,neighborhood,borough)
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto2)
map_toronto2

This has a lot fewer points so it should use up less of my Foursquare calls.
Now I will set up my Foursquare ID and secrect. Then I will run a test call.

In [27]:
CLIENT_ID = 'MAYXPAXJWFBOZSPGAOON2UQO4X5GOKIGFPCU4VGOOBSYMLYF'
CLIENT_SECRET = 'GWFLAS3QDKPURQPYQGCA4P515TZFBLCNBKB1IOG53VXWOUNZ' # your Foursquare Secret
VERSION = '20190802' # Foursquare API version

In [28]:
toronto_df.loc[0, 'Neighbourhood']
neighborhood_latitude = toronto_df.loc[0, 'Latitude'] 
neighborhood_longitude = toronto_df.loc[0, 'Longitude'] 

neighborhood_name = toronto_df.loc[0, 'Neighbourhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront,Regent Park are 43.6542599, -79.3606359.


I am using the first post code to test an make sure it is working. I will create a Get request URL and Send the GET request.

In [64]:
Limit=25
radius=100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    Limit)


In [65]:
results=requests.get(url).json()

In [66]:
results

{'meta': {'code': 200, 'requestId': '5d484b2b86bc49002ceaa45b'},
 'response': {'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.6551599009, 'lng': -79.35939430002972},
   'sw': {'lat': 43.653359899099996, 'lng': -79.36187749997029}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50760559e4b0e8c7babe2497',
       'name': 'Body Blitz Spa East',
       'location': {'address': '497 King Street East',
        'crossStreet': 'btwn Sackville St and Sumach St',
        'lat': 43.65473505045365,
        'lng': -79.35987433132891,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65473505045365,
          'lng': -79.35987433132891

In [67]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [68]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Body Blitz Spa East,Spa,43.654735,-79.359874
1,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
2,Henrietta Lane,Cocktail Bar,43.653911,-79.361395
3,King Deli Cafe,Sandwich Place,43.653906,-79.361147
4,Massimo's Kitchen Studio,Italian Restaurant,43.654770,-79.359698


Now we can see the venues close to the first postol code

In [69]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


This returned the max number of venues so I will increase the max when I run the code for all the postol codes.

In [70]:
LIMIT=50
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [71]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
The Danforth West,Riverdale
Design Exchange,Toronto Dominion Centre
Brockton,Exhibition Place,Parkdale Village
The Beaches West,India Bazaar
Commerce Court,Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North,Forest Hill West
High Park,The Junction South
North Toronto West
The Annex,North Midtown,Yorkville
Parkdale,Roncesvalles
Davisville
Harbord,University of Toronto
Runnymede,Swansea
Moore Park,Summerhill East
Chinatown,Grange Park,Kensington Market
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city


In [72]:
print(toronto_venues.shape)
toronto_venues.head()

(1166, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront,Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront,Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront,Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront,Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Harbourfront,Regent Park",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


Now we can check the number of venues for each neighborhood group of each postol code

In [73]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",50,50,50,50,50,50
Berczy Park,50,50,50,50,50,50
"Brockton,Exhibition Place,Parkdale Village",21,21,21,21,21,21
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",17,17,17,17,17,17
"Cabbagetown,St. James Town",48,48,48,48,48,48
Central Bay Street,50,50,50,50,50,50
"Chinatown,Grange Park,Kensington Market",50,50,50,50,50,50
Christie,16,16,16,16,16,16


Now we will look at each Neighborhood groups of each postol code and print the top 5 venues for each Neighbor hood

In [74]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

In [75]:
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
toronto_onehot.shape

(1166, 213)

Group rows by neighborhoods and take the mean of the frequency of occurrence of each category

In [77]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060000,0.000000,...,0.00,0.00,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.00,0.00
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.00,0.00
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00,0.00
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,...,0.00,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00,0.00
5,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020833,0.000000,...,0.00,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00,0.00
6,Central Bay Street,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,...,0.00,0.00,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.02,0.00
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.02,0.000000,0.00000,0.00,0.060000,0.000000,0.060000,0.02,0.00
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00,0.00
9,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.02,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.020000,0.00,0.02


In [78]:
toronto_grouped.shape

(38, 213)

Each postol code along with the top 5 most common venues. I am leaving the names of the neighborhood.

In [79]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.06
2  American Restaurant  0.06
3           Steakhouse  0.06
4            Gastropub  0.04


----Berczy Park----
          venue  freq
0   Coffee Shop  0.08
1  Cocktail Bar  0.06
2          Café  0.04
3      Beer Bar  0.04
4   Cheese Shop  0.04


----Brockton,Exhibition Place,Parkdale Village----
               venue  freq
0        Coffee Shop  0.10
1     Breakfast Spot  0.10
2               Café  0.10
3      Grocery Store  0.05
4  Convenience Store  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1          Comic Shop  0.06
2       Garden Center  0.06
3          Smoke Shop  0.06
4      Farmers Market  0.06


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airpor

Create a dataframe for this information

In [80]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [81]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Steakhouse,American Restaurant,Café,Bar,Gastropub,Pizza Place,Hotel,Asian Restaurant,Monument / Landmark
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Steakhouse,Beer Bar,Farmers Market,Cheese Shop,Café,Seafood Restaurant,Breakfast Spot
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Café,Breakfast Spot,Climbing Gym,Bar,Italian Restaurant,Intersection,Caribbean Restaurant,Falafel Restaurant,Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Park,Brewery,Recording Studio,Fast Food Restaurant,Farmers Market,Auto Workshop,Burrito Place,Comic Shop,Pizza Place
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Plane,Harbor / Marina,Coffee Shop,Sculpture Garden,Boutique,Bar,Boat or Ferry


I am adding the postol code back into the dataframe

In [47]:
#neighborhoods_venues_sorted['Postcode']=toronto_df['Postcode']

In [86]:
#neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postcode
0,"Adelaide,King,Richmond",Coffee Shop,Steakhouse,American Restaurant,Café,Hotel,Bar,Gastropub,Pizza Place,Asian Restaurant,Jazz Club,M5A
1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Bakery,Steakhouse,Beer Bar,Cheese Shop,Café,Seafood Restaurant,Fish Market,M5B
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Café,Breakfast Spot,Grocery Store,Intersection,Performing Arts Venue,Pet Store,Convenience Store,Climbing Gym,Caribbean Restaurant,M5C
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Spa,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Moving Target,Comic Shop,Park,M4E
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Sculpture Garden,Boat or Ferry,Coffee Shop,Plane,Boutique,Airport Gate,M5E


Now I will use the k-means to cluster the postol codes into 5 clusters

In [82]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [89]:
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_df
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0,Coffee Shop,Bakery,Café,Park,Breakfast Spot,Pub,Mexican Restaurant,Restaurant,Shoe Store,Brewery
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Coffee Shop,Café,Ramen Restaurant,Clothing Store,Bookstore,Theater,Fast Food Restaurant,Hotel,Spa,Cosmetics Shop
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Gastropub,Restaurant,Italian Restaurant,Hotel,Farmers Market,Park,Cocktail Bar,Japanese Restaurant,Café
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Health Food Store,Pub,Wings Joint,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Dive Bar,Discount Store
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Bakery,Steakhouse,Beer Bar,Farmers Market,Cheese Shop,Café,Seafood Restaurant,Breakfast Spot


One more map to visualize the clusters

In [90]:
#This is copied directly from the lab as I do not know what color and options to use.


# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine the clusters

In [91]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0,2] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,"Harbourfront,Regent Park",0,Coffee Shop,Bakery,Café,Park,Breakfast Spot,Pub,Mexican Restaurant,Restaurant,Shoe Store,Brewery
1,M5B,"Ryerson,Garden District",0,Coffee Shop,Café,Ramen Restaurant,Clothing Store,Bookstore,Theater,Fast Food Restaurant,Hotel,Spa,Cosmetics Shop
2,M5C,St. James Town,0,Coffee Shop,Gastropub,Restaurant,Italian Restaurant,Hotel,Farmers Market,Park,Cocktail Bar,Japanese Restaurant,Café
3,M4E,The Beaches,0,Trail,Health Food Store,Pub,Wings Joint,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Dive Bar,Discount Store
4,M5E,Berczy Park,0,Coffee Shop,Cocktail Bar,Bakery,Steakhouse,Beer Bar,Farmers Market,Cheese Shop,Café,Seafood Restaurant,Breakfast Spot
5,M5G,Central Bay Street,0,Coffee Shop,Burger Joint,Ice Cream Shop,Café,Spa,Bubble Tea Shop,Italian Restaurant,Chinese Restaurant,Modern European Restaurant,Seafood Restaurant
6,M6G,Christie,0,Grocery Store,Café,Park,Coffee Shop,Convenience Store,Diner,Italian Restaurant,Baby Store,Athletics & Sports,Restaurant
7,M5H,"Adelaide,King,Richmond",0,Coffee Shop,Steakhouse,American Restaurant,Café,Bar,Gastropub,Pizza Place,Hotel,Asian Restaurant,Monument / Landmark
8,M6H,"Dovercourt Village,Dufferin",0,Supermarket,Pharmacy,Bakery,Music Venue,Café,Middle Eastern Restaurant,Bank,Bar,Art Gallery,Park
9,M5J,"Harbourfront East,Toronto Islands,Union Station",0,Coffee Shop,Park,Hotel,Bar,Brewery,Pizza Place,Aquarium,Café,Japanese Restaurant,Skating Rink


In [92]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0,2] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,M4N,Lawrence Park,1,Bus Line,Park,Swim School,Wings Joint,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Dive Bar,Discount Store,Diner


In [93]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0,2] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,M5N,Roselawn,2,Garden,Ice Cream Shop,Wings Joint,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Dive Bar,Discount Store,Diner


In [94]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[0,2] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,M4T,"Moore Park,Summerhill East",3,Playground,Gym,Park,Summer Camp,Wings Joint,Cupcake Shop,Dumpling Restaurant,Dog Run,Dive Bar,Discount Store


In [95]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[0,2] + list(range(5, toronto_merged.shape[1]))]]

,Postcode,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,M5P,"Forest Hill North,Forest Hill West",4,Park,Trail,Jewelry Store,Sushi Restaurant,Wings Joint,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Dive Bar
32,M4W,Rosedale,4,Park,Playground,Trail,Building,Wings Joint,Cupcake Shop,Dumpling Restaurant,Dog Run,Dive Bar,Discount Store


I am not sure why my 1st cluster has all of the data. I tried adjusting my call by decreasing the radius because I though there migh be a lot of overlap with the locations I was getting.